# Open AI modellek finomhangolása

Ez a jegyzet az Open AI [Fine Tuning](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) dokumentációjában található aktuális útmutatáson alapul.

A finomhangolás javítja az alapmodellek teljesítményét az adott alkalmazásodhoz, mivel további, az adott felhasználási esetre vagy szcenárióra vonatkozó adatokkal és kontextussal tanítjuk újra a modellt. Fontos megjegyezni, hogy a prompt engineering technikák, mint például a _few shot learning_ vagy a _retrieval augmented generation_, lehetővé teszik, hogy a promptot releváns adatokkal egészítsük ki a jobb minőség érdekében. Ezeket a megközelítéseket azonban korlátozza a célzott alapmodell maximális token ablakmérete.

A finomhangolás során magát a modellt tanítjuk újra a szükséges adatokkal (így sokkal több példát tudunk használni, mint amennyi a maximális token ablakba beleférne) – és egy _egyedi_ verziót hozunk létre, amelynek már nincs szüksége példákra az előrejelzés során. Ez nemcsak a prompt tervezésének hatékonyságát növeli (nagyobb szabadságunk van a token ablak más célokra való felhasználásában), hanem akár a költségeinket is csökkentheti (mivel kevesebb tokent kell elküldenünk a modellnek előrejelzéskor).

A finomhangolás 4 lépésből áll:
1. Az oktatási adatok előkészítése és feltöltése.
1. A tanítási folyamat elindítása, hogy elkészüljön a finomhangolt modell.
1. A finomhangolt modell kiértékelése és a minőség javítása.
1. A finomhangolt modell bevetése előrejelzéshez, ha elégedettek vagyunk az eredménnyel.

Fontos, hogy nem minden alapmodell támogatja a finomhangolást – a legfrissebb információkért [nézd meg az OpenAI dokumentációját](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst). Korábban már finomhangolt modellt is lehet tovább finomhangolni. Ebben az útmutatóban a `gpt-35-turbo` modellt fogjuk használni finomhangolásra.

---


### 1.1. lépés: Készítsd elő az adathalmazodat

Építsünk egy chatbotot, amely segít megérteni az elemek periódusos rendszerét, és limerick formájában válaszol az egyes elemekkel kapcsolatos kérdésekre. Ebben az egyszerű útmutatóban csak egy adathalmazt hozunk létre, amely néhány mintaválaszt tartalmaz, hogy bemutassa az elvárt adatformátumot. Valós alkalmazás esetén sokkal több példát kellene összegyűjtened. Ha létezik nyílt adathalmaz az adott területhez, azt is felhasználhatod, és átalakíthatod a finomhangoláshoz.

Mivel a `gpt-35-turbo`-ra koncentrálunk, és egyetlen válaszlépést (chat completion) várunk, a példákat [ebben a javasolt formátumban](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) készíthetjük el, amely megfelel az OpenAI chat completion elvárásainak. Ha többlépéses beszélgetést szeretnél, akkor a [többlépéses példák formátumát](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst) kellene használnod, amely tartalmaz egy `weight` paramétert is, hogy jelezd, mely üzeneteket használja (vagy ne használja) a finomhangolás során.

Itt az egyszerűbb, egyválaszos formátumot használjuk az útmutatóban. Az adatokat [jsonl formátumban](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) adjuk meg, ahol minden sor egy rekord, JSON objektumként. Az alábbi részlet 2 rekordot mutat mintaként – a teljes mintakészletet (10 példa) a [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) fájlban találod, amit ebben a finomhangolási útmutatóban használunk. **Megjegyzés:** Minden rekordot _egy_ sorban kell megadni (nem lehet több sorba tördelni, ahogy a formázott JSON fájloknál szokásos).

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

Valós alkalmazás esetén sokkal nagyobb példakészletre lesz szükséged a jó eredményekhez – a válaszok minősége és a finomhangolás ideje/költsége között kell majd egyensúlyozni. Mi most kis mintát használunk, hogy gyorsan bemutathassuk a folyamatot. Egy összetettebb finomhangolási útmutatóért nézd meg [ezt az OpenAI Cookbook példát](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst).


### 1.2. lépés: Adatkészlet feltöltése

Töltsd fel az adatokat a Files API segítségével [ahogy itt le van írva](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). Fontos, hogy a kód futtatása előtt az alábbi lépéseket már elvégezd:
 - Telepítetted az `openai` Python csomagot (ügyelj rá, hogy legalább 0.28.0-s vagy újabb verziót használj a legfrissebb funkciókhoz)
 - Beállítottad az `OPENAI_API_KEY` környezeti változót a saját OpenAI API kulcsoddal
További információért nézd meg a tanfolyamhoz tartozó [Beállítási útmutatót](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst).

Most futtasd a kódot, hogy létrehozz egy feltöltésre alkalmas fájlt a helyi JSONL fájlodból.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


### 2.1. lépés: Hozd létre a finomhangolási feladatot az SDK-val


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


### 2.2. lépés: A feladat állapotának ellenőrzése

Íme néhány dolog, amit a `client.fine_tuning.jobs` API-val megtehetsz:
- `client.fine_tuning.jobs.list(limit=<n>)` – Az utolsó n finomhangolási feladat listázása
- `client.fine_tuning.jobs.retrieve(<job_id>)` – Egy adott finomhangolási feladat részleteinek lekérdezése
- `client.fine_tuning.jobs.cancel(<job_id>)` – Egy finomhangolási feladat leállítása
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` – Legfeljebb n esemény listázása a feladathoz kapcsolódóan
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

A folyamat első lépése a _tanítófájl érvényesítése_, hogy biztosan megfelelő formátumban legyenek az adatok.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


### 2.3. lépés: Kövesd nyomon az eseményeket a haladás figyeléséhez


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### 2.4. lépés: Állapot megtekintése az OpenAI irányítópulton


A státuszt megtekintheted az OpenAI weboldalán is, a _Fine-tuning_ szekcióban. Itt láthatod az aktuális feladat állapotát, valamint nyomon követheted a korábbi futtatások történetét is. Ebben a képernyőképen látható, hogy az előző futás sikertelen volt, míg a második sikeresen lefutott. Ennek az az oka, hogy az első futásnál egy hibásan formázott rekordokat tartalmazó JSON fájlt használtak – miután ezt kijavították, a második futás már sikeresen befejeződött, és a modell elérhetővé vált a használathoz.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba7e3f73a201f8712fae3cea1c08f7c7f12ca469c06d234122.hu.png)


A státusz üzeneteket és a mérőszámokat is megtekintheti, ha lejjebb görget a vizuális irányítópulton, ahogy az alábbi képen látható:

| Üzenetek | Mérőszámok |
|:---|:---|
| ![Üzenetek](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b3a706a66f66bf5007c379cd9219cfb74cb30c0b04b90c4c8.hu.png) |  ![Mérőszámok](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a652299584ab181536a6cfb67691a897a518b6c7a2aa0a17f1a30d.hu.png)|


### 3.1. lépés: Azonosító lekérése és a finomhangolt modell tesztelése kódban


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


### 3.2. lépés: Finomhangolt modell betöltése és tesztelése a Playgroundban

Most kétféleképpen is kipróbálhatod a finomhangolt modellt. Először is, látogass el a Playground felületre, és a Models legördülő menüből válaszd ki az újonnan finomhangolt modelledet a felsorolt lehetőségek közül. A másik lehetőség, hogy a Fine-tuning panelen megjelenő "Playground" opciót használod (lásd a fenti képernyőképet), amely elindítja az alábbi _összehasonlító_ nézetet, ahol egymás mellett láthatod az alap és a finomhangolt modell verzióit, így gyorsan értékelheted az eredményeket.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-compare.56e06f0ad8922016497d39ced3d84ea296eec89073503f2bf346ec9718f913b5.hu.png)

Egyszerűen töltsd ki a rendszerkörnyezetet, amit a tanító adataidban használtál, és add meg a tesztkérdésedet. Látni fogod, hogy mindkét oldalon ugyanaz a környezet és kérdés jelenik meg. Futtasd le az összehasonlítást, és megfigyelheted a különbséget a kimenetek között. _Figyeld meg, hogy a finomhangolt modell a példáidban megadott formátumban adja vissza a választ, míg az alapmodell csak a rendszerpromptot követi._

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-launch.5a26495c983c6350c227e05700a47a89002d132949a56fa4ff37f266ebe997b2.hu.png)

Azt is észre fogod venni, hogy az összehasonlítás mindkét modellhez megmutatja a tokenek számát, valamint az inferencia idejét. **Ez a konkrét példa egy egyszerűsített eset, amely a folyamatot mutatja be, de nem tükröz valós adathalmazt vagy helyzetet.** Előfordulhat, hogy mindkét minta ugyanannyi tokent tartalmaz (a rendszerkörnyezet és a felhasználói kérdés azonos), viszont a finomhangolt modellnél hosszabb az inferencia ideje (egyedi modell).

Valós helyzetekben nem ilyen játékpéldát fogsz használni, hanem valódi adatokkal (például termékkatalógus ügyfélszolgálathoz) hangolod a modellt, ahol a válaszok minősége sokkal szembetűnőbb lesz. Ilyen esetben az alapmodellel hasonló minőségű válasz eléréséhez több egyedi prompt tervezésre lesz szükség, ami növeli a tokenhasználatot és az inferencia idejét is. _Ha ki szeretnéd próbálni, nézd meg az OpenAI Cookbook finomhangolási példáit a kezdéshez._



---

**Jogi nyilatkozat**:  
Ez a dokumentum AI fordítási szolgáltatás, a [Co-op Translator](https://github.com/Azure/co-op-translator) segítségével készült. Bár törekszünk a pontosságra, kérjük, vegye figyelembe, hogy az automatikus fordítások hibákat vagy pontatlanságokat tartalmazhatnak. Az eredeti, eredeti nyelvű dokumentum tekintendő hiteles forrásnak. Kritikus információk esetén javasoljuk a professzionális, emberi fordítást. Nem vállalunk felelősséget a fordítás használatából eredő félreértésekért vagy téves értelmezésekért.
